# Group Number : 11

### Group Members

#### Raj Rajeshwari Bajre           23PGAI0093
#### Darshana Nitin Mandhana  23PGAI0089
#### Anurag Pandey                     23PGAI0059
#### Girish Khule                          23PGAI0057

# Importing all libraries

In [1]:
import json
import nltk
import re
import pandas as pd
from collections import Counter
import requests
from bs4 import BeautifulSoup
import spacy
from nltk import pos_tag
from nltk import RegexpParser
from fractions import Fraction
from transformers import pipeline
import tkinter as tk
import customtkinter
from functools import partial 
import pyttsx3

# Fetching data of recipe

### The scarpping was done from foodnetwork website - https://www.foodnetwork.com/

In [2]:
recipe = {}
global_dish = ''

In [3]:
def get_recipie_details(dish):
    global global_dish 
    global_dish = dish
    ingredient_list = []
    step_list = []
    level = '-'
    total_time = '-'
    prep_time = '-'
    cook_time = '-'
    servings = '-'

    dish_to_search = ''
    split_text = dish.split(' ')
    for i in split_text:
        dish_to_search+=i
        dish_to_search+='-'

    #searching dish on the foodnetwork website 
    URL = f"https://www.foodnetwork.com/search/{dish_to_search}"
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'html.parser')

    #Extracting all recipe label URL for the dish
    recipe_result_list = soup.find_all('div',{'class':'m-MediaBlock__m-TextWrap'})
    recipe_links = []
    for i in recipe_result_list:
        if i.find('span',{'class':'m-Info__a-SubHeadline'}) != None:
            is_result_recipe = True if i.find('span',{'class':'m-Info__a-SubHeadline'}).text.strip() == 'Recipe' else False
            if is_result_recipe:
                recipe_link_tag = i.find('h3',{'class':'m-MediaBlock__a-Headline'})
                recipe_links.append('https:' + recipe_link_tag.find('a')['href'])            
    recipe_link_url = recipe_links[0]


    URL = recipe_link_url
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'html.parser')

    time_serving_details = soup.find_all('span',{'class':'o-RecipeInfo__a-Description'})
    if time_serving_details != []:
        #inactive time case
        if(len(time_serving_details)/2 >= 6):
            print(time_serving_details[0]['class'])
            level = time_serving_details[0].text.strip()
            total_time = time_serving_details[1].text.strip()
            prep_time = time_serving_details[2].text.strip() + '+' + time_serving_details[3].text.strip()
            cook_time = time_serving_details[4].text.strip()
            servings = time_serving_details[5].text.strip()
        #prep and cook time case
        elif(len(time_serving_details)/2 >= 5):
            level = time_serving_details[0].text.strip()
            total_time = time_serving_details[1].text.strip()
            prep_time = time_serving_details[2].text.strip()
            cook_time = time_serving_details[3].text.strip()
            servings = time_serving_details[4].text.strip()
        else:
            level = time_serving_details[0].text.strip()
            total_time = time_serving_details[1].text.strip()
            cook_time = time_serving_details[2].text.strip()
            servings = time_serving_details[3].text.strip()

    #Extracting Ingredients
    ingredients = soup.find_all('span',{'class':'o-Ingredients__a-Ingredient--CheckboxLabel'})
    ingredient = [ingredient.text.strip() for ingredient in ingredients]
    ingredient_list.extend(ingredient)
    if len(ingredient_list) != 0:
        del ingredient_list[0]

    #Extracting Steps
    steps = soup.find_all('li',{'class':'o-Method__m-Step'})
    step = [step.text.strip() for step in steps]
    step_list.extend(step)


    #Recipe Dictionary

    recipe[dish] = {'ingredient':ingredient_list,'step':step_list,'level':level,'total_time':total_time,'prep_time':prep_time,'cook_time':cook_time,'servings':servings}

    return recipe
    

In [4]:
recipe = get_recipie_details('drumsticks') #4 servings
global_dish = 'drumsticks'

# POS Tagging

In [5]:
def pos_tagging_ingredient(recipe,global_dish):
    pos_ingredient_dict = {}
    for ingredient in recipe[global_dish]['ingredient']:
        split_ingredient = ingredient.split()
        if global_dish not in pos_ingredient_dict:
            pos_ingredient_dict[global_dish] = []
        pos_ingredient_dict[global_dish].append(pos_tag(split_ingredient))
        
    return pos_ingredient_dict

In [6]:
s = 'how much olive oil'
tag=pos_tag(s.split(' '))
tag

[('how', 'WRB'), ('much', 'JJ'), ('olive', 'JJ'), ('oil', 'NN')]

# Parsing the recipe

In [7]:
#regex for ingredientQueries: how much, amount, quantity, how many, count of 
ingredientQueries=['much','amount','quantity','many','count']
search_nouns=[]
for ta in tag:
    if ta[1]=='NN':
        search_nouns.append(ta[0])
print(search_nouns)           

['oil']


In [8]:
#with POS-Searching the noun from the query in ingredient
se=0
if search_nouns[se] in ingredientQueries:
    while se<len(search_nouns):
        for ing in ingredient:
            if search_nouns[se] in ing.split():
                print(ing)                
        se+=1

In [9]:
ingredient = recipe[global_dish]['ingredient']

In [10]:
ingredient

['8 drumsticks',
 '2 tablespoons vegetable oil',
 '1 teaspoon chili powder',
 '1 teaspoon garlic powder',
 'Salt and freshly ground black pepper',
 '1/4 cup (1/2 stick) unsalted butter, melted',
 '1/4 cup hot sauce',
 '1 tablespoon BBQ sauce']

In [11]:
se=0
while se<len(s.split()):
    for ing in ingredient:
        if s.split()[se] in ing.split():
            print(ing)                
    se+=1

2 tablespoons vegetable oil


In [12]:
def IngredientQuery(s):
    dish = global_dish
    pos_ingredient_dict = {}
    for ingredient in recipe[dish]['ingredient']:
        split_ingredient = ingredient.split()
        if dish not in pos_ingredient_dict:
            pos_ingredient_dict[dish] = []
        pos_ingredient_dict[dish].append(pos_tag(split_ingredient)) 
        
    ingredientQueries=['much','amount','quantity','many','count','step'] #Generic query words related to ingredient questions
    
    #let us extract all the nouns from the ingredient list and add them to the ingredientQueries list
    ingredient=recipe[dish]['ingredient'] 
    for ingr in ingredient:
        wordIng=ingr.split()
        tagIng=pos_tag(wordIng)
        for tagIn in tagIng:
            if tagIn[1]=='NN' or tagIn[1]=='NNS' or tagIn[1]=='NNP': #filter on the basis of Noun 
                ingredientQueries.append(tagIn[0])
                
    #Now let us extract all the nouns from the asked question
    search_nouns=[]
    tag=pos_tag(s.split(' '))
    for ta in tag:
        if ta[1]=='NN':
            search_nouns.append(ta[0])
    
    #IF the nouns from question matches the nouns from ingredientQueries list, return the ingredient with description
    se=0
    while se<len(search_nouns):
        if search_nouns[se] in ingredientQueries:
            for ing in ingredient:
                if search_nouns[se] in ing.split():
                    return(ing)                
        se+=1


In [13]:
qa_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


# Scaling Ingredients

In [14]:
def scale_ingredients(global_dish,scale_ingredient_by,pos_ingredient_dict):
    pos_ingredient_dict_scaled = pos_ingredient_dict
    counter1 = 0
    for element in pos_ingredient_dict[global_dish]:
        counter2 = 0
        for tags in element:
            if tags[1] == 'CD':
                pos_ingredient_dict_scaled[global_dish][counter1][counter2] = [float(Fraction(tags[0]))*scale_ingredient_by,'CD']
            counter2+=1
        counter1+=1
    return pos_ingredient_dict_scaled


In [15]:
def print_ingredients(global_dish,ingredients):
    for element in ingredients[global_dish]:
        ingredient = ''
        for tags in element:
            ingredient += str(tags[0]) + ' '
        print(ingredient)

In [16]:
pos_ingredient_dict = {}
pos_ingredient_dict_scaled = {}
scale_ingredient_by = 12#int(input("How many servings do you need?" ))#enter 12
print()
servings = int(recipe[global_dish]['servings'][0])

if(servings != 0):
    pos_ingredient_dict = pos_tagging_ingredient(recipe,global_dish)

    #before scaling
    print_ingredients(global_dish,pos_ingredient_dict)
    print()

    pos_ingredient_dict_scaled = scale_ingredients(global_dish,(scale_ingredient_by/servings),pos_ingredient_dict)
    #after scaling
    print_ingredients(global_dish,pos_ingredient_dict_scaled)



8 drumsticks 
2 tablespoons vegetable oil 
1 teaspoon chili powder 
1 teaspoon garlic powder 
Salt and freshly ground black pepper 
1/4 cup (1/2 stick) unsalted butter, melted 
1/4 cup hot sauce 
1 tablespoon BBQ sauce 

24.0 drumsticks 
6.0 tablespoons vegetable oil 
3.0 teaspoon chili powder 
3.0 teaspoon garlic powder 
Salt and freshly ground black pepper 
0.75 cup (1/2 stick) unsalted butter, melted 
0.75 cup hot sauce 
3.0 tablespoon BBQ sauce 


# User Interface

### We used tkinter to build the user interface

### Note : We tried to integrate audio, but tkinter itself is too heavy so we had dead kernel issue due to which we have commented the audio code.

In [17]:
import tkinter as tk
import customtkinter

customtkinter.set_appearance_mode("dark")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"

app = customtkinter.CTk()
app.geometry("1120x1000")
app.title("My Cook Book!")

step_pointer = 0
recipe = {}

# engineio = pyttsx3.init()
# engineio.say("Welcome to the cook book")
# engineio.runAndWait()


def button_callback():
    dish = entry_1.get()
    global recipe
    recipe = get_recipie_details(dish)
    
    title = "Ingredients for " + dish
    label_2.configure(text = title)
    
    textbox.configure(state="normal")
    textbox.delete("1.0","end")
    show_ingredients(dish)
    show_ingredients_label.configure(state = "normal")
    show_step.configure(state = "normal")
    scale_ingredients_button.configure(state = "normal")
    
    #filling labels
    level_label.configure(text = recipe[dish]['level'])
    total_time_label.configure(text = recipe[dish]['total_time'])
    cook_time_label.configure(text = recipe[dish]['cook_time'])
    prep_time_label.configure(text = recipe[dish]['prep_time'])
    servings_label.configure(text = recipe[dish]['servings'][0])

def show_ingredients_callback():
    dish = entry_1.get()
    title = "Ingredients for " + dish
    label_2.configure(text = title)
    
    textbox.configure(state="normal")
    textbox.delete("1.0","end")
    show_ingredients(dish)
    show_step.configure(state = "normal")
    scale_ingredients_button.configure(state = "normal")
#     for ingredient in recipe[global_dish]['ingredient']:
#        speak("Ingrediens List")
#        speak(ingredient)
    
def show_ingredients(dish):
    for ingredient in recipe[dish]['ingredient']:
       textbox.insert("0.0", ingredient + '\n')
    textbox.configure(state="disabled")
    question_ingredient.configure(state = 'normal')
    
def show_steps():
    global step_pointer
    step_pointer = 0
    textbox.configure(state="normal")
    textbox.delete("1.0","end")
    dish = entry_1.get()
    label_2.configure(text = "Steps for " + dish)
    textbox.insert("0.0", recipe[dish]['step'][0])
    next_step.configure(state="normal")
    previous_step.configure(state="normal")
    question.configure(state="normal")
    question_ingredient.configure(state = "normal")
    textbox.configure(state="disabled")
#     engineio.say(recipe[dish]['step'][step_pointer])
    
def next_step_callback():
    global step_pointer
    textbox.configure(state="normal")
    textbox.delete("1.0","end")
    dish = entry_1.get()
        
    if step_pointer >= 0 and step_pointer < len(recipe[dish]['step'])-1:
        step_pointer += 1
        textbox.insert("0.0", recipe[dish]['step'][step_pointer])
    else: 
        textbox.insert("0.0", "Your recipe is ready!")
#         engineio.say("Your recipe is ready!")
        if step_pointer == len(recipe[dish]['step'])-1:
            step_pointer += 1
    textbox.configure(state="disabled")
#     engineio.say(recipe[dish]['step'][step_pointer])
    
def previous_step_callback():
    global step_pointer
    
    dish = entry_1.get()
    
    if step_pointer > 0:
        textbox.configure(state="normal")
        textbox.delete("1.0","end")
        step_pointer -= 1
        if step_pointer >= 0 and step_pointer <= len(recipe[dish]['step']) -1:
            textbox.insert("0.0", recipe[dish]['step'][step_pointer])
        textbox.configure(state="disabled")
#         engineio.say(recipe[dish]['step'][step_pointer])

def question_callback():
    dish = entry_1.get()
    dialog = customtkinter.CTkInputDialog(text="Type in your question", title="Question Dialog Box")
    question = dialog.get_input()
    context = recipe[dish]['step'][step_pointer]
    result = qa_pipeline(question = question, context = context)
    label_ans.configure(text = result['answer'] )
    
def question_for_ingredient_callback():
    dish = entry_1.get()
    dialog = customtkinter.CTkInputDialog(text="Type in your question", title="Question Dialog Box")
    question = dialog.get_input()
    context = ''
    
    ans = IngredientQuery(question)
    if ans == None:  
        for i in recipe[dish]['ingredient']:
            context = context + i + ' '
        result = qa_pipeline(question = question, context = context)
        label_ans_ingredient.configure(text = result['answer'] )
    else : 
        label_ans_ingredient.configure(text = ans )
    
    

def speak(text):
    engineio = pyttsx3.init()
    voices = engineio.getProperty('voices')
    engineio.setProperty('rate', 160)
    engineio.setProperty('voice',voices[0].id)
    engineio.say(text)
    engineio.runAndWait()

def question_callback_for_any_question():
    dish = entry_1.get()
    dialog = customtkinter.CTkInputDialog(text="Type in your question", title="Question Dialog Box")
    question = dialog.get_input()
    context = ''
    for i in recipe[dish]['ingredient']:
        context = context + i + ' '
    for i in recipe[dish]['step']:
        context = context + i + ' '
    result = qa_pipeline(question = question, context = context)
    label_ans.configure(text = result['answer'] )   
    
def change_appearance_mode_event(new_appearance_mode: str):
    customtkinter.set_appearance_mode(new_appearance_mode)

def change_scaling_event(new_scaling: str):
    new_scaling_float = int(new_scaling.replace("%", "")) / 100
    customtkinter.set_widget_scaling(new_scaling_float)
    
def scale_ingredient_callback():
    dish = entry_1.get()
    dialog = customtkinter.CTkInputDialog(text="How many servings do you want?", title="Scale Ingredients")
    scale_ingredient_by = int(dialog.get_input())
    
    pos_ingredient_dict = {}
    pos_ingredient_dict_scaled = {}
    servings = int(recipe[global_dish]['servings'][0])

    if(servings != 0):
        pos_ingredient_dict = pos_tagging_ingredient(recipe,global_dish)
    
    pos_ingredient_dict_scaled = scale_ingredients(global_dish,(scale_ingredient_by/servings),pos_ingredient_dict)
    
    label_2.configure(text = "Sacled Ingredients for " + dish)
    textbox.configure(state="normal")
    textbox.delete("1.0","end")
        
    for element in pos_ingredient_dict_scaled[global_dish]:
        ingredient = ''
        for tags in element:
            ingredient += str(tags[0]) + ' '
        textbox.insert("0.0", ingredient + '\n')
    textbox.configure(state="disabled")
    
    
    
#Question Frame
question_frame = customtkinter.CTkFrame(master=app)
question_frame.grid(pady=20, padx=(55,0), row = 0, column = 0, sticky = "news")

#Label for Question Frame
label_frame = customtkinter.CTkLabel(master=question_frame, text = "Query Section for Recipe", font=customtkinter.CTkFont(size=20, weight="bold"))
label_frame.grid(pady=(10,20), padx=10)

#Question Button for Ingredient 
question_ingredient = customtkinter.CTkButton(master=question_frame, text="Ingredient Question",command=question_for_ingredient_callback)
question_ingredient.grid(pady=(240,0), padx=10)
question_ingredient.configure(state = "disabled")

#Label for answering questions
label_ans_ingredient = customtkinter.CTkLabel(master=question_frame, text = "", font=customtkinter.CTkFont(size=15, weight="normal"))
label_ans_ingredient.grid(pady=(10,20), padx=10)

#Question Button for Steps
question = customtkinter.CTkButton(master=question_frame, text="Step question",command=question_callback)
question.grid(pady=20, padx=10)
question.configure(state = "disabled")

#Label for answering questions
label_ans = customtkinter.CTkLabel(master=question_frame, text = "", font=customtkinter.CTkFont(size=15, weight="normal"))
label_ans.grid(pady=10, padx=10)




    
#Main frame
frame_1 = customtkinter.CTkFrame(master=app)
frame_1.grid(pady=20, padx=30, row = 0, column = 1, sticky = "news")

label_1 = customtkinter.CTkLabel(master=frame_1, text="Enter the recipe you want to search below",font=customtkinter.CTkFont(size=20, weight="bold"))
label_1.grid(pady=10, padx=10)

entry_1 = customtkinter.CTkEntry(master=frame_1, placeholder_text="Enter the recipe name here",width = 200)
entry_1.grid(pady=10, padx=10)

button_1 = customtkinter.CTkButton(master=frame_1, text="Search",command=button_callback)
button_1.grid(pady=10, padx=10)

label_2 = customtkinter.CTkLabel(master=frame_1, text="",font=customtkinter.CTkFont(size=20, weight="bold"))
label_2.grid(pady=10, padx=10)

textbox = customtkinter.CTkTextbox(master=frame_1, width=500, height=350, font=customtkinter.CTkFont(size=15, weight="normal"))
textbox.grid(pady=10, padx=10)
textbox.configure(state="disabled")

show_ingredients_label = customtkinter.CTkButton(master=frame_1, text="Show Ingredients",command=show_ingredients_callback)
show_ingredients_label.grid(pady=10, padx=10)
show_ingredients_label.configure(state = "disabled")

show_step = customtkinter.CTkButton(master=frame_1, text="Show Steps",command=show_steps)
show_step.grid(pady=10, padx=10)
show_step.configure(state = "disabled")

next_step = customtkinter.CTkButton(master=frame_1, text="Next Step",command=next_step_callback)
next_step.grid(pady=10, padx=10)
next_step.configure(state = "disabled")

previous_step = customtkinter.CTkButton(master=frame_1, text="Previous Step",command=previous_step_callback)
previous_step.grid(pady=10, padx=10)
previous_step.configure(state = "disabled")




#Sidebar Frame
sidebar_frame = customtkinter.CTkFrame(master=app)
sidebar_frame.grid(pady=20, padx=(0,30), row = 0, column = 2, sticky = "news")

logo_label = customtkinter.CTkLabel(master = sidebar_frame, text="Recipe Details", font=customtkinter.CTkFont(size=20, weight="bold"))
logo_label.grid(pady=10, padx=20)

#Level Label
logo_label = customtkinter.CTkLabel(master = sidebar_frame, text="Level", font=customtkinter.CTkFont(size=15, weight="bold"))
logo_label.grid(pady=(20,0), padx=10)

level_label = customtkinter.CTkLabel(master=sidebar_frame, text = "", font=customtkinter.CTkFont(size=15, weight="normal"))
level_label.grid(pady=0, padx=10)

#Total Time Label
logo_label = customtkinter.CTkLabel(master = sidebar_frame, text="Total Time", font=customtkinter.CTkFont(size=15, weight="bold"))
logo_label.grid(pady=(20,0), padx=10)

total_time_label = customtkinter.CTkLabel(master=sidebar_frame, text = "", font=customtkinter.CTkFont(size=15, weight="normal"))
total_time_label.grid(pady=0, padx=10)

#Prep Time Label
logo_label = customtkinter.CTkLabel(master = sidebar_frame, text="Prep Time", font=customtkinter.CTkFont(size=15, weight="bold"))
logo_label.grid(pady=(20,0), padx=10)

prep_time_label = customtkinter.CTkLabel(master=sidebar_frame, text = "", font=customtkinter.CTkFont(size=15, weight="normal"))
prep_time_label.grid(pady=0, padx=10)

#Cook Time Label
logo_label = customtkinter.CTkLabel(master = sidebar_frame, text="Cook Time", font=customtkinter.CTkFont(size=15, weight="bold"))
logo_label.grid(pady=(20,0), padx=10)

cook_time_label = customtkinter.CTkLabel(master=sidebar_frame, text = "", font=customtkinter.CTkFont(size=15, weight="normal"))
cook_time_label.grid(pady=0, padx=10)

#Servings Label
logo_label = customtkinter.CTkLabel(master = sidebar_frame, text="Servings", font=customtkinter.CTkFont(size=15, weight="bold"))
logo_label.grid(pady=(20,0), padx=10)

servings_label = customtkinter.CTkLabel(master=sidebar_frame, text = "", font=customtkinter.CTkFont(size=15, weight="normal"))
servings_label.grid(pady=0, padx=10)

#Scale ingredients
scale_label = customtkinter.CTkLabel(master = sidebar_frame, text="No of Servings", anchor="w")
scale_label.grid(pady=(20,0), padx=10)
scale_ingredients_button = customtkinter.CTkButton(master=sidebar_frame, text="Scale",command=scale_ingredient_callback)
scale_ingredients_button.grid(pady=(10,20), padx=10)
scale_ingredients_button.configure(state = 'disabled')

#Others
appearance_mode_label = customtkinter.CTkLabel(master = sidebar_frame, text="Appearance Mode", anchor="w")
appearance_mode_label.grid(pady=(20,0), padx=10)
appearance_mode_optionemenu = customtkinter.CTkOptionMenu(master = sidebar_frame, values=["Dark", "Light", "System"],command=change_appearance_mode_event)
appearance_mode_optionemenu.grid(pady=(10,20), padx=10)

scaling_label = customtkinter.CTkLabel(master = sidebar_frame, text="UI Scaling", anchor="w")
scaling_label.grid(pady=(20,0), padx=10)
scaling_optionemenu = customtkinter.CTkOptionMenu(master = sidebar_frame, values=["100%", "80%", "90%", "110%", "120%"], command=change_scaling_event)
scaling_optionemenu.grid(pady=(10,20), padx=10)


app.mainloop()

['o-RecipeInfo__a-Description']


# Audio of the whole recipe

In [ ]:
# !pip install pywin32
# !pip install pyttsx3
# !pip install python-engineio

import pyttsx3

engineio = pyttsx3.init()
voices = engineio.getProperty('voices')
engineio.setProperty('rate', 160)    # Aquí puedes seleccionar la velocidad de la voz
engineio.setProperty('voice',voices[0].id)

def speak(text):
    engineio.say(text)
    engineio.runAndWait()

speak("Ingrediens List for " + global_dish)
for ingredient in recipe[global_dish]['ingredient']:
       speak(ingredient)
   
speak("Step List for " + global_dish)
for step in recipe[global_dish]['step']:
       speak(step)

# Voice

In [ ]:
# pip install pipwin
# !pipwin install pyaudio
# !pip install SpeechRecognition
# !pip install pyaudio

import speech_recognition as sr
listener = sr.Recognizer()
with sr.Microphone() as source:
    voice = listener.listen(source)
    command = listener.recognize_google(voice)
    command = command.lower()